In [1]:
from pyrh import Robinhood
from datetime import datetime as dt
import datetime
import numpy as np
import sched
import pandas_ta as pta
import time
import pyotp

In [2]:
#Login Info
login = open("C:/Users/Molina/Documents/Untitled Folder/algo/login").read().splitlines()
email = login[0]
pw = login[1]
key = login[2]
code = login[3]
totp = pyotp.TOTP(key).now()

In [3]:
rh = Robinhood()
rh.login(username = email, password = pw, qr_code = key)

True

In [4]:
#set Variables
enteredTrade = False
fastPeriod = 2
slowPeriod = 10
ticker = 'AAPL'

In [5]:
def ema(closePrices, period, smoothing = 2):
    #returns last ema and current ema
    
    ema = [sum(closePrices[-period:]) / period]
    
    for price in closePrices[-period:]:
        ema.append((price * (smoothing / (1 + period))) + ema[-1] * (1 - (smoothing / (1 + period))))
        
    return ema[-2], ema[-1]

In [6]:
#Schedule so we can check on the price changes each minute
s = sched.scheduler(time.time, time.sleep)

In [7]:
def run(sc): 
    
    global enteredTrade
    global fastPeriod
    global slowPeriod
    global ticker
    
    print("Getting historical quotes")
    buyPrice = -1
    priceDiff = 0
    stopLoss = 1
    enteredTrade = False 
    
    # Get 5 minute bar data for ticker's stock
    historical_quotes = rh.get_historical_quotes(ticker, "5minute", "day")
    closePrices = []
    
    #Calculate support and resistance
    currentIndex = 0
    currentSupport  = 0
    currentResistance = 0
    
    for key in historical_quotes["results"][0]["historicals"]:
        time = datetime.datetime.now()
        print ("The time is now: = %s:%s:%s" % (time.hour, time.minute, time.second))
        
        if (currentIndex >= len(historical_quotes["results"][0]["historicals"]) - (100)):
            if (currentIndex >= (slowPeriod - 1) and dt.strptime(key['begins_at'], '%Y-%m-%dT%H:%M:%SZ').minute == 0):
                currentSupport = 0
                currentResistance = 0
                print("Resetting support and resistance")
            if(float(key['close_price']) < currentSupport or currentSupport == 0):
               currentSupport = float(key['close_price'])
               print("Current Support is : ")
               print(currentSupport)
            if(float(key['close_price']) > currentResistance):
               currentResistance = float(key['close_price'])
               print("Current Resistance is : ")
               print(currentResistance)
            closePrices.append(float(key['close_price']))
        currentIndex += 1
    if (len(closePrices) > (slowPeriod)):
        #Calculate EMAs
        lastEMASlow, presEMASlow = ema(closePrices, slowPeriod)
        lastEMAFast, presEMAFast = ema(closePrices, fastPeriod)
        instrument = rh.instruments(ticker)[0]
        if buyPrice != -1:
            priceDiff = closePrices[i] - buyPrice
        
        #stoploss if stock dips under the stop loss, sell
        if priceDiff < -stopLoss and enteredTrade == True:
            print("Selling! Stock dipped under the stop loss")
            rh.place_sell_order(instrument, 1)
            enteredTrade = False
            
        #If fast EMA crosses above slow EMA Buys and sets stop loss to be 5% of that value
        if lastEMASlow > lastEMAFast and presEMASlow < presEMAFast and float(key['close_price']) <= currentSupport and not enteredTrade:
            print("Buying! Slow EMA crossed above fast EMA")
            buyprice = closePrices[-1]
            stopLoss = buyPrice * .1
            rh.place_buy_order(instrument, 1)
            enteredTrade = True
            
        #If fast EMA crosses below slow EMA Sell
        if lastEMASlow < lastEMAFast and presEMASlow > presEMAFast and float(key['close_price']) >= currentResistance and currentResistance > 0 and enteredTrade:
            print("Selling! Slow EMA crossed above fast EMA")
            rh.place_sell_order(instrument, 1)
            enteredTrade = False
        print('Current slow EMA: ')
        print(presEMASlow)
        print('Current fast EMA: ')
        print(presEMAFast)
    #call this method again every 5 minutes for new price changes
    s.enter(300, 1, run, (sc,))

In [ ]:
s.enter(1, 1, run, (s,))
s.run()

Getting historical quotes
The time is now: = 12:12:50
Current Support is : 
176.0
Current Resistance is : 
176.0
The time is now: = 12:12:50
Current Resistance is : 
176.36
The time is now: = 12:12:50
The time is now: = 12:12:50
The time is now: = 12:12:50
The time is now: = 12:12:50
The time is now: = 12:12:50
Current Support is : 
175.67
The time is now: = 12:12:50
The time is now: = 12:12:50
Current Support is : 
175.0954
The time is now: = 12:12:50
The time is now: = 12:12:50
The time is now: = 12:12:50
The time is now: = 12:12:50
The time is now: = 12:12:50
The time is now: = 12:12:50
The time is now: = 12:12:50
The time is now: = 12:12:50
Current Support is : 
175.07
The time is now: = 12:12:50
The time is now: = 12:12:50
Resetting support and resistance
Current Support is : 
175.07
Current Resistance is : 
175.07
The time is now: = 12:12:50
Current Support is : 
174.94
The time is now: = 12:12:50
Current Support is : 
174.72
The time is now: = 12:12:50
Current Support is : 
174.

Getting historical quotes
The time is now: = 12:27:51
Current Support is : 
176.0
Current Resistance is : 
176.0
The time is now: = 12:27:51
Current Resistance is : 
176.36
The time is now: = 12:27:51
The time is now: = 12:27:51
The time is now: = 12:27:51
The time is now: = 12:27:51
The time is now: = 12:27:51
Current Support is : 
175.67
The time is now: = 12:27:51
The time is now: = 12:27:51
Current Support is : 
175.0954
The time is now: = 12:27:51
The time is now: = 12:27:51
The time is now: = 12:27:51
The time is now: = 12:27:51
The time is now: = 12:27:51
The time is now: = 12:27:51
The time is now: = 12:27:51
The time is now: = 12:27:51
Current Support is : 
175.07
The time is now: = 12:27:51
The time is now: = 12:27:51
Resetting support and resistance
Current Support is : 
175.07
Current Resistance is : 
175.07
The time is now: = 12:27:51
Current Support is : 
174.94
The time is now: = 12:27:51
Current Support is : 
174.72
The time is now: = 12:27:51
Current Support is : 
174.

Getting historical quotes
The time is now: = 12:42:52
Current Support is : 
176.0201
Current Resistance is : 
176.0201
The time is now: = 12:42:52
Current Resistance is : 
176.36
The time is now: = 12:42:52
The time is now: = 12:42:52
The time is now: = 12:42:52
Current Support is : 
176.01
The time is now: = 12:42:52
The time is now: = 12:42:52
Current Support is : 
175.67
The time is now: = 12:42:52
The time is now: = 12:42:52
Current Support is : 
175.14
The time is now: = 12:42:52
Current Support is : 
175.135
The time is now: = 12:42:52
The time is now: = 12:42:52
The time is now: = 12:42:52
The time is now: = 12:42:52
The time is now: = 12:42:52
The time is now: = 12:42:52
The time is now: = 12:42:52
Current Support is : 
175.045
The time is now: = 12:42:52
The time is now: = 12:42:52
Resetting support and resistance
Current Support is : 
175.0671
Current Resistance is : 
175.0671
The time is now: = 12:42:52
Current Support is : 
174.96
The time is now: = 12:42:52
Current Support

Getting historical quotes
The time is now: = 12:57:54
Current Support is : 
176.0
Current Resistance is : 
176.0
The time is now: = 12:57:54
Current Resistance is : 
176.36
The time is now: = 12:57:54
The time is now: = 12:57:54
The time is now: = 12:57:54
The time is now: = 12:57:54
The time is now: = 12:57:54
Current Support is : 
175.67
The time is now: = 12:57:54
The time is now: = 12:57:54
Current Support is : 
175.0954
The time is now: = 12:57:54
The time is now: = 12:57:54
The time is now: = 12:57:54
The time is now: = 12:57:54
The time is now: = 12:57:54
The time is now: = 12:57:54
The time is now: = 12:57:54
The time is now: = 12:57:54
Current Support is : 
175.07
The time is now: = 12:57:54
The time is now: = 12:57:54
Resetting support and resistance
Current Support is : 
175.07
Current Resistance is : 
175.07
The time is now: = 12:57:54
Current Support is : 
174.94
The time is now: = 12:57:54
Current Support is : 
174.72
The time is now: = 12:57:54
Current Support is : 
174.

Getting historical quotes
The time is now: = 13:12:55
Current Support is : 
176.0201
Current Resistance is : 
176.0201
The time is now: = 13:12:55
Current Resistance is : 
176.36
The time is now: = 13:12:55
The time is now: = 13:12:55
The time is now: = 13:12:55
Current Support is : 
176.01
The time is now: = 13:12:55
The time is now: = 13:12:55
Current Support is : 
175.67
The time is now: = 13:12:55
The time is now: = 13:12:55
Current Support is : 
175.14
The time is now: = 13:12:55
Current Support is : 
175.135
The time is now: = 13:12:55
The time is now: = 13:12:55
The time is now: = 13:12:55
The time is now: = 13:12:55
The time is now: = 13:12:55
The time is now: = 13:12:55
The time is now: = 13:12:55
Current Support is : 
175.045
The time is now: = 13:12:55
The time is now: = 13:12:55
Resetting support and resistance
Current Support is : 
175.0671
Current Resistance is : 
175.0671
The time is now: = 13:12:55
Current Support is : 
174.96
The time is now: = 13:12:55
Current Support

...

Getting historical quotes
The time is now: = 15:13:7
Current Support is : 
176.0
Current Resistance is : 
176.0
The time is now: = 15:13:7
Current Resistance is : 
176.36
The time is now: = 15:13:7
The time is now: = 15:13:7
The time is now: = 15:13:7
The time is now: = 15:13:7
The time is now: = 15:13:7
Current Support is : 
175.67
The time is now: = 15:13:7
The time is now: = 15:13:7
Current Support is : 
175.0954
The time is now: = 15:13:7
The time is now: = 15:13:7
The time is now: = 15:13:7
The time is now: = 15:13:7
The time is now: = 15:13:7
The time is now: = 15:13:7
The time is now: = 15:13:7
The time is now: = 15:13:7
Current Support is : 
175.07
The time is now: = 15:13:7
The time is now: = 15:13:7
Resetting support and resistance
Current Support is : 
175.07
Current Resistance is : 
175.07
The time is now: = 15:13:7
Current Support is : 
174.94
The time is now: = 15:13:7
Current Support is : 
174.72
The time is now: = 15:13:7
Current Support is : 
174.69
The time is now: = 

Getting historical quotes
The time is now: = 15:28:9
Current Support is : 
176.0
Current Resistance is : 
176.0
The time is now: = 15:28:9
Current Resistance is : 
176.36
The time is now: = 15:28:9
The time is now: = 15:28:9
The time is now: = 15:28:9
The time is now: = 15:28:9
The time is now: = 15:28:9
Current Support is : 
175.67
The time is now: = 15:28:9
The time is now: = 15:28:9
Current Support is : 
175.0954
The time is now: = 15:28:9
The time is now: = 15:28:9
The time is now: = 15:28:9
The time is now: = 15:28:9
The time is now: = 15:28:9
The time is now: = 15:28:9
The time is now: = 15:28:9
The time is now: = 15:28:9
Current Support is : 
175.07
The time is now: = 15:28:9
The time is now: = 15:28:9
Resetting support and resistance
Current Support is : 
175.07
Current Resistance is : 
175.07
The time is now: = 15:28:9
Current Support is : 
174.94
The time is now: = 15:28:9
Current Support is : 
174.72
The time is now: = 15:28:9
Current Support is : 
174.69
The time is now: = 

Getting historical quotes
The time is now: = 15:43:10
Current Support is : 
176.0201
Current Resistance is : 
176.0201
The time is now: = 15:43:10
Current Resistance is : 
176.36
The time is now: = 15:43:10
The time is now: = 15:43:10
The time is now: = 15:43:10
Current Support is : 
176.01
The time is now: = 15:43:10
The time is now: = 15:43:10
Current Support is : 
175.67
The time is now: = 15:43:10
The time is now: = 15:43:10
Current Support is : 
175.14
The time is now: = 15:43:10
Current Support is : 
175.135
The time is now: = 15:43:10
The time is now: = 15:43:10
The time is now: = 15:43:10
The time is now: = 15:43:10
The time is now: = 15:43:10
The time is now: = 15:43:10
The time is now: = 15:43:10
Current Support is : 
175.045
The time is now: = 15:43:10
The time is now: = 15:43:10
Resetting support and resistance
Current Support is : 
175.0671
Current Resistance is : 
175.0671
The time is now: = 15:43:10
Current Support is : 
174.96
The time is now: = 15:43:10
Current Support

Getting historical quotes
The time is now: = 15:58:12
Current Support is : 
176.0201
Current Resistance is : 
176.0201
The time is now: = 15:58:12
Current Resistance is : 
176.36
The time is now: = 15:58:12
The time is now: = 15:58:12
The time is now: = 15:58:12
Current Support is : 
176.01
The time is now: = 15:58:12
The time is now: = 15:58:12
Current Support is : 
175.67
The time is now: = 15:58:12
The time is now: = 15:58:12
Current Support is : 
175.14
The time is now: = 15:58:12
Current Support is : 
175.135
The time is now: = 15:58:12
The time is now: = 15:58:12
The time is now: = 15:58:12
The time is now: = 15:58:12
The time is now: = 15:58:12
The time is now: = 15:58:12
The time is now: = 15:58:12
Current Support is : 
175.045
The time is now: = 15:58:12
The time is now: = 15:58:12
Resetting support and resistance
Current Support is : 
175.0671
Current Resistance is : 
175.0671
The time is now: = 15:58:12
Current Support is : 
174.96
The time is now: = 15:58:12
Current Support

Getting historical quotes
The time is now: = 16:13:13
Current Support is : 
176.0201
Current Resistance is : 
176.0201
The time is now: = 16:13:13
Current Resistance is : 
176.36
The time is now: = 16:13:13
The time is now: = 16:13:13
The time is now: = 16:13:13
Current Support is : 
176.01
The time is now: = 16:13:13
The time is now: = 16:13:13
Current Support is : 
175.67
The time is now: = 16:13:13
The time is now: = 16:13:13
Current Support is : 
175.14
The time is now: = 16:13:13
Current Support is : 
175.135
The time is now: = 16:13:13
The time is now: = 16:13:13
The time is now: = 16:13:13
The time is now: = 16:13:13
The time is now: = 16:13:13
The time is now: = 16:13:13
The time is now: = 16:13:13
Current Support is : 
175.045
The time is now: = 16:13:13
The time is now: = 16:13:13
Resetting support and resistance
Current Support is : 
175.0671
Current Resistance is : 
175.0671
The time is now: = 16:13:13
Current Support is : 
174.96
The time is now: = 16:13:13
Current Support